In [ ]:
import os
import sys
from datetime import datetime

sys.path.insert(0, "../../../../..")
sys.path.insert(0, "../../../../../../acquire")

from HUGS.Processing import search
from HUGS.Client import Process, Search, Retrieve
from Acquire.ObjectStore import datetime_to_string
from Acquire.Client import User, Drive, Service, PAR, Authorisation, StorageCreds

from HUGS.Client import Search
from HUGS.Util import get_datapath
from HUGS.Interface import Interface

import json

import ipyleaflet as ipl
import ipywidgets as ipw

# Autoreload modules before executing code, useful during development
%load_ext autoreload
%autoreload 2

In [ ]:
base_url= "https://hugs.acquire-aaai.com/t"
search = Search(service_url=base_url)

search_terms = ["ch4"]
locations = []

noaa_results = search.search(search_terms=search_terms, locations=locations, data_type="NOAA")

In [ ]:
box_layout = ipw.Layout(display='flex', flex_flow='column', align_items='center', width='50%')

guide_text = ipw.HTML(value="<font size>To plot data from the sites shown on the map below please click on the label and then click download. A plotting box will then appear below the map. Two sites may be compared by clicking on both sites and ensuring their codes are visible at the top right of the map before clicking download. Please note that as data is being pulled from the object store, it may take a few moments for the plotting box to appear.</font>")
guide_box = ipw.VBox(children=[guide_text], layout=box_layout)

In [ ]:
# Load in the ACRG site data
acrg_json = "../site_data/acrg_with_locations.json"
with open(acrg_json, "r") as f:
    acrg_sites = json.load(f)
    

i = Interface()
noaa_map, fig_widgets = i.mapbox_notebook(search_results=noaa_results, site_data=acrg_sites, center=[-5,37], zoom=2, data_type="NOAA")

noaa_map.layout.width = '50%'
noaa_map.layout.height = '400px'

edgar_image_path = "Emissions_Americas.png"

edgar_layer = ipl.ImageOverlay(url=edgar_image_path, bounds=((-60,-140), (55,-30)))
noaa_map.add_layer(edgar_layer)

noaa_box = ipw.VBox(children=[noaa_map, fig_widgets])

In [ ]:
search_terms = ["co2"]
locations = []

eurocom_results = search.search(search_terms=search_terms, locations=locations, data_type="EUROCOM")

In [ ]:
eurocom_map, eurowidgets = i.mapbox_notebook(search_results=eurocom_results, site_data=acrg_sites, center=[55,2], zoom=4, data_type="EUROCOM")

eurocom_map.layout.width = '50%'
eurocom_map.layout.height = '400px'
                                             
eurobox = ipw.VBox(children=[eurocom_map, eurowidgets])

In [ ]:
center = [51.506815, -0.56]
zoom = 10

map_london = ipl.Map(center=center, zoom=zoom)
map_london.layout.width = '50%'
map_london.layout.height = '400px'

positron_layer = ipl.basemap_to_tiles(ipl.basemaps.CartoDB.Positron)

map_london.add_layer(positron_layer)


marker_legend = ipw.HTML(value="<img src='marker-icon-blue.png'>   Current site<br><img src='marker-icon-green.png'>   Future site")
marker_control = ipl.WidgetControl(widget=marker_legend, position="topright")

lghg_sites = "../site_data/lghg_sites.json"
map_london.add_control(marker_control)

with open(lghg_sites, "r") as f:
    lghg_data = json.load(f)
    

for site in lghg_data["current"]:
    curr_site = lghg_data["current"][site]
    
    lat = curr_site["latitude"]
    long = curr_site["longitude"]
    site_name = curr_site["long_name"]
    
    marker = ipl.Marker(location=(lat, long), draggable=False)
    marker.popup = ipw.HTML(value=f"Site: {site_name} ({site})")
    map_london.add_layer(marker)
    
for site in lghg_data["future"]:
    fut_site = lghg_data["future"][site]
    
    lat = fut_site["latitude"]
    long = fut_site["longitude"]
    site_name = fut_site["long_name"]
    
    # Here we want a green icon
    icon = ipl.Icon(icon_url='marker-icon-green.png', icon_size=[25, 40], icon_anchor=[12,15])
    
    marker = ipl.Marker(location=(lat, long), draggable=False, icon=icon)
    marker.popup = ipw.HTML(value=f"Site: {site_name} ({site})")
    map_london.add_layer(marker)
    
london_footprint = "high_res_london_block_inferno_50p.png"

footprint = ipl.ImageOverlay(url=london_footprint, bounds=((51.2458, -1.259), (51.7092, 0.17389)))
map_london.add_layer(footprint)

In [ ]:
box_layout = ipw.Layout(display='flex',
                flex_flow='column',
                align_items='center',
                width='100%')

text_layout = ipw.Layout(display='flex',
                flex_flow='column',
                align_items='center',
                width='50%')

centre_layout = ipw.Layout(align_items='center')

text = "Figure 2.  Examples of a) Global Measurements locations from the NOAA \
network, b) selected nation / continental ICOS stations and c) current and \
planned urban measurement sites from the LondonGHG project, currently\
available for analysis on the HUGS platform. GHGs currently on the platform \
include CO2, CH4, N2O, halocarbons and related tracers (e.g. CO). Panel \
(a) also shows an overlay of emissions estimates from the EDGAR dataset. \
A footprint over Greater London is overlaid on panel (c). These images are \
screenshots from a Jupyter notebook hosted on the HUGS platform."

figure_text = ipw.HTML(value=text, layout=text_layout)


complete = ipw.VBox(children=[guide_box, noaa_box, eurobox, map_london, figure_text])

hidden_box = ipw.VBox(children=[guide_box], layout=ipw.Layout(max_width="10%", visibility="hidden"))

side = ipw.HBox(children=[hidden_box, complete])


In [ ]:
side